# Examining Physics Using the Interpolation Method

This notebook demonstrates the final phase of Spyral, solving. To run this notebook, the previous three phases (point cloud, cluster, estimate) of Spyral *must* have been run on the data. Now that we have generated our clusters and estimated the physical observables of interest we are ready to initiate the solving phase of the analysis, where we attempt to extract the exact physics observables by fitting solutions of the equations of motion to the data. There are several approaches to solving in this application, and this notebook looks at what is referred to as the interpolation method. The interpolation method works by pre-generating a bunch of solutions to the ODE's and then interpolation on these solutions to try and fit the data. It has the advantage of being very fast; the ODE's only ever need to be solved once, and then all the remaining calculation is just simple (bi)linear interpolation.

First let's take care of all of our imports.

Note: this notebook assumes you've already done the work of generating the interpolation scheme. This is done by simply running phase 4 in the main application with an interpolation scheme defined. This can take some time (20-30 min depending on the coarse-ness of the grid).

In [ ]:
import sys
sys.path.append('..')
from spyral.core.cluster import Cluster
from spyral.interpolate.track_interpolator import create_interpolator
from spyral.core.config import load_config
from spyral.core.workspace import Workspace
from spyral.core.particle_id import load_particle_id
from spyral.solvers.solver_interp import fit_model_interp, Guess, interpolate_trajectory

from spyral_utils.nuclear.target import GasTarget, load_target
from spyral_utils.nuclear import NuclearDataMap

import polars as pl
import numpy as np
import h5py as h5
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Now with all of our code imported we will setup the configuration, loading our config file and creating all of our data structures

In [ ]:
config = load_config('../local_config.json')
ws = Workspace(config.workspace)
nuc_map = NuclearDataMap()
pid = load_particle_id(ws.get_gate_file_path(config.solver.particle_id_filename), nuc_map)
target = load_target(Path(config.solver.gas_data_path), nuc_map)
if not isinstance(target, GasTarget):
    raise Exception('Target error!')
track_path = ws.get_track_file_path(pid.nucleus, target)
tracks = create_interpolator(track_path)

Now we'll pick a data file (the min run number defined in the config by default) and load up the associated physics estimates. By default we select a random row in the estimates (a random cluster/trajectory) but you can always set the row to a hardcoded number to do some testing of parameters.

In [ ]:
run_number = config.run.run_min
cluster_file = h5.File(ws.get_cluster_file_path(run_number))
estimate_df = pl.scan_parquet(ws.get_estimate_file_path_parquet(run_number))
estimate_gated = estimate_df.filter(pl.struct(['dEdx', 'brho']).map_batches(pid.cut.is_cols_inside))\
        .collect() \
        .to_dict()
cluster_group = cluster_file['cluster']
nrows = len(estimate_gated['event'])
row = np.random.randint(0, nrows)
# row = 1198
print(f'row: {row}')
event = estimate_gated['event'][row]
cluster_index = estimate_gated['cluster_index'][row]
print(f'event: {event}')
print(f'cluster index: {cluster_index}')
event_group = cluster_group[f'event_{event}']
local_cluster = event_group[f'cluster_{cluster_index}']
print(f'Direction: {estimate_gated["direction"][row]}')
cluster = Cluster(event, local_cluster.attrs['label'], local_cluster['cloud'][:].copy())

With our cluster and estimated observables loaded, we are ready to fit to the data. We setup our Guess object from our estimates and then pass that along to the fit_model function. Sometimes this will return None when a given trajectory has estimates that are outside the interpolation table (these typically correspond to bad events). If this happens a error will occur. Simply re-run the notebook until the a good event is randomly selected. Note that the first time you run this block it might take a couple of seconds. This is because the interpolation method uses a just-in-time compiler (jit) to speed up the calculations. The first time you call the code, the code gets compiled (resulting in a slowdown). But everytime the code is called after that, the compiled program is used, resulting in enormus performance gains.

In [ ]:
guess = Guess(estimate_gated['polar'][row], estimate_gated['azimuthal'][row], estimate_gated['brho'][row], estimate_gated['vertex_x'][row], estimate_gated['vertex_y'][row], estimate_gated['vertex_z'][row])
print(guess)
result = fit_model_interp(cluster, guess, tracks, pid.nucleus)
if result is None:
    print('Guess outside of interpolation range!')
best_fit_trajectory_xy = interpolate_trajectory(result, tracks, pid.nucleus).interpolate(cluster.data[:, 2]*0.001)
cluster.data[:, :3] *= 0.001

If a good event was chosen, you should see above a print out of the fit results. Key values are the chi-square (which should be small) and the variable values, which are the fitted observables. Also important are the correlations, which tell you if any of the parameters are co-dependent. If two parameters have a correlation of 1.0 they are basically degenerate to the fitter, which is very bad.

We can also plot the results of the fit against the data to vizualize the performance

In [ ]:
fig = make_subplots(2, 2, subplot_titles=["XY Projection", "XZ Projection", "YZ Projection"], specs=[[{"rowspan": 2}, {}],[None, {}]])
fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 0],
        y=cluster.data[:, 1],
        mode="markers", 
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory_xy[:, 0],
        y=best_fit_trajectory_xy[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_x"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex"
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=best_fit_trajectory_xy[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_x"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False,
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=best_fit_trajectory_xy[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False
    ),
    row=2,
    col=2
)

fig["layout"]["xaxis1"]["title"] = "X (m)"
fig["layout"]["xaxis1"]["range"] = [-0.3, 0.3]
fig["layout"]["yaxis1"]["title"] = "Y (m)"
fig["layout"]["yaxis1"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis2"]["title"] = "Z (m)"
fig["layout"]["xaxis2"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis2"]["title"] = "X (m)"
fig["layout"]["yaxis2"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis3"]["title"] = "Z (m)"
fig["layout"]["xaxis3"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis3"]["title"] = "Y (m)"
fig["layout"]["yaxis3"]["range"] = [-0.3, 0.3]

fig.update_layout(
    width=1450,
    height=725
)

Hopefully you see a nice fit to the data! If the fit looks bad, there are several things to check. First is the particle ID gate; if the wrong particle group is selected, the fit will fail spectacularly. Another is the coarse-ness of the interpolation scheme. If there are too few bins in the polar angle or the particle kinetic energy, the interpolation may not generate good values. Finally, it is also good to make sure that the target gas is correctly defined with the right pressure and chemistry.